$s$: CARを計算できたPRの文書集合、$s_1 (\subset s)$: CARが高かったPRの文書集合、$s_2 (:= s - s_1)$: CARが高くなかったPRの文書集合とする。

単語$t$に対して、tf-idf値を以下のように定義。

$$
{\rm tf}(t, s_1) = \frac{df(t, s_1)}{|s_1|}
$$

$$
{\rm idf}(t) = \log\frac{|s|}{df(t,s)}
$$

$$
{\rm df}(t, x) = \left|\{d\mid d\ni t, d\in x\}\right|
$$

$$
{\rm tf}\cdot{\rm idf}(t, s_1) = {\rm tf}(t, s_1)\cdot {\rm idf}(t)
$$

tf-idfが高い単語から並べる。

In [1]:
## 入力データのファイル名
pr_mor_fname = "../fix_input_data/pressrelease_all_normalized.csv.mor"   # MeCab で形態素に分解済みのプレスリリース本文
car_sig_fname = "car_exactT_alpha0.1_sig"
car_not_sig_fname = "car_exactT_alpha0.1_not_sig"

In [2]:
## import
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math
from scipy import stats
from multiprocessing import Pool
from collections import Counter

## データの読み込み

In [3]:
## CAR。tf-idf 計算は、car_sig と car_all = car_sig + car_not_sig の間で行う
## CAR が計算できたものだけを取り扱う点、PR ごとに CAR の平均値を取らない点(後述)が前回と異なる。

car_sig = pd.read_table(car_sig_fname, sep = '\t')   # 有意なもの
car_not_sig = pd.read_table(car_not_sig_fname, sep = '\t')   # 有意でないもの
car_sig[0:3]

,article_id,pr_type,comp_code,car,t_stat
0,NIKPRLRSP040809_18022003,01: Product,2593,0.079042,1.582459
1,NIKPRLRSP045017_21042003,04: Alliance,7599,0.120927,1.583770
2,NIKPRLRSP050445_10072003,01: Product,7911,0.104491,2.325275


In [ ]:
pr_mor = pd.DataFrame()
with open(pr_mor_fname, 'r') as f:
    for line in f:
        article_id, mor = line.strip().split('\t')
        pr_mor = pr_mor.append([[article_id, mor]], ignore_index = True)
pr_mor.columns = ["article_id", "pr_mor"]

In [113]:
## 全 PR を形態素分解したもの
pr_mor = pd.read_table(pr_mor_fname, sep = '\t', header = None, names = ['article_id', 'pr_mor'])
pr_mor[0:3]

CParserError: Error tokenizing data. C error: out of memory

## df, idf, tf, tf-idf の計算

In [68]:
id_set_sig = set(car_sig["article_id"])
id_set_not_sig = set(car_not_sig["article_id"])
id_set_all = id_set_sig.union(id_set_not_sig)

num_sig = len(id_set_sig)   # equal to |s_1|
num_all = len(id_set_all)   # equal to |s|
print(num_sig, len(id_set_not_sig), len(id_set_sig.intersection(id_set_not_sig)), num_all)

706 24397 108 24995


-> 有意な CAR を持つ PR 706個のうち、108個が有意でない(他の企業とのマッチングによる) CAR も同時に持っている

TODO: これの扱い (現在は有意な PR に含めている; PR ごとに CAR の平均を取ること(前回はこれを採用していた)はしたくない)

In [64]:
pr_sig = pr_mor[[x in id_set_sig for x in list(pr_mor["article_id"])]]   # pr_mor から、article_id が car_sig に含まれるものだけを抽出
pr_all = pr_mor[[x in id_set_all for x in list(pr_mor["article_id"])]]

In [65]:
pr_sig

,article_id,pr_mor
10,NIKPRLRSP040809_18022003,発表 日 : 2003年 2月18日 ” 101 ” は おいしい 烏龍茶 の しるし 。 ...


In [66]:
pr_all

,article_id,pr_mor
1,NIKPRLRSP038060_07012003,発表 日 : 2003年 1月7日 中国 で 単音 ・ 和音 着信メロディ 配信 サービス ...
10,NIKPRLRSP040809_18022003,発表 日 : 2003年 2月18日 ” 101 ” は おいしい 烏龍茶 の しるし 。 ...


In [73]:
pr_as_word_set_sig = [set(x.split(' ')) for x in list(pr_sig["pr_mor"])]   # 有意な PR を形態素分解した単語集合からなるリスト
pr_as_word_set_all = [set(x.split(' ')) for x in list(pr_all["pr_mor"])]   # 全部

In [94]:
max_count = 4#100000   # very sensitive for input data
min_count = 4#100   # very sensitive for input data

word_count = Counter()
for mor in list(pr_all["pr_mor"]):
    for w in mor.split(' '):
        word_count[w] += 1
        
## TODO: 単語の純粋な出現回数ではなく、出現したPRの数でフィルターするべき？

In [95]:
words = set()   # filtered set of words to be used for tf-idf calculation
for w, c in word_count.items():
    if min_count <= c and c <= max_count:
        words.add(w)

In [97]:
words

{'まし',
 'れ',
 'メロリン',
 '中',
 '今回',
 '単音',
 '味わい',
 '品種',
 '概要',
 '約',
 '美麗',
 '茶',
 '製法',
 '鈴',
 '黄金色'}

In [102]:
def calc_pr_num_with_word_in_sig(word):   # これをword set に付いて並列化
    count = 0
    for mor in pr_as_word_set_sig:
        if word in mor:
            count += 1
    return (word, count)

def calc_pr_num_with_word_in_all(word):   # これをword set に付いて並列化
    count = 0
    for mor in pr_as_word_set_all:
        if word in mor:
            count += 1
    return (word, count)

# TODO: 1つにまとめる

In [103]:
exe_pool = Pool(2)
df_sig = Counter()
for ret in exe_pool.imap(calc_pr_num_with_word_in_sig, words):
    df_sig[ret[0]] = ret[1]

In [104]:
df_sig

Counter({'まし': 1,
         'れ': 1,
         'メロリン': 0,
         '中': 1,
         '今回': 1,
         '単音': 0,
         '味わい': 1,
         '品種': 1,
         '概要': 1,
         '約': 0,
         '美麗': 0,
         '茶': 1,
         '製法': 1,
         '鈴': 0,
         '黄金色': 1})

In [105]:
exe_pool = Pool(2)
df_all = Counter()
for ret in exe_pool.imap(calc_pr_num_with_word_in_all, words):
    df_all[ret[0]] = ret[1]

In [106]:
df_all

Counter({'まし': 2,
         'れ': 2,
         'メロリン': 1,
         '中': 1,
         '今回': 2,
         '単音': 1,
         '味わい': 1,
         '品種': 1,
         '概要': 2,
         '約': 1,
         '美麗': 1,
         '茶': 1,
         '製法': 1,
         '鈴': 1,
         '黄金色': 1})

In [108]:
tfidf = {}
for w in words:
    tfidf[w] = float(df_sig[w]) / num_sig * math.log(float(num_all) / df_all[w], 2)

In [109]:
tfidf

{'まし': 0.019276702417239215,
 'れ': 0.019276702417239215,
 'メロリン': 0.0,
 '中': 0.020693133012140066,
 '今回': 0.019276702417239215,
 '単音': 0.0,
 '味わい': 0.020693133012140066,
 '品種': 0.020693133012140066,
 '概要': 0.019276702417239215,
 '約': 0.0,
 '美麗': 0.0,
 '茶': 0.020693133012140066,
 '製法': 0.020693133012140066,
 '鈴': 0.0,
 '黄金色': 0.020693133012140066}

In [111]:
print(str(num_all) + "\t" + str(num_sig))   # # PRs in all PR set, partial set
for data in sorted(tfidf.items(), key=lambda x: x[1], reverse = True):                                             
    print('\t'.join([data[0], str(data[1]), str(df_all[data[0]]), str(df_sig[data[0]])]))   # word, tf-idf, # PR in all PR with the word, # PR in partial set
    
# TODO: statistical test

24995	706
製法	0.020693133012140066	1	1
中	0.020693133012140066	1	1
黄金色	0.020693133012140066	1	1
茶	0.020693133012140066	1	1
品種	0.020693133012140066	1	1
味わい	0.020693133012140066	1	1
まし	0.019276702417239215	2	1
れ	0.019276702417239215	2	1
今回	0.019276702417239215	2	1
概要	0.019276702417239215	2	1
単音	0.0	1	0
鈴	0.0	1	0
約	0.0	1	0
美麗	0.0	1	0
メロリン	0.0	1	0
